In [1]:
import math
import pandas as pd
from lxml import etree
not_found_string = "N/A"
frc_county_loc = "/home/will/Documents/Projects/frc_counties/data/frc_teams_counties.csv"
svg_loc = "/home/will/Documents/Projects/frc_counties/data/counties.svg"
target_loc = "frc_county_density.svg"
min_color_val = 25

In [2]:
# new library funcs for modifying svg file
def writeNewSVGWithColorDictionary(color_dict, tooltips_dict = None):
    tree = etree.parse(svg_loc)
    root = tree.getroot()
    paths = root[2]
    for path in paths:
        path_county_id = path.attrib["id"]
        if path_county_id not in color_dict:
            continue
        path.attrib["style"] = "fill: " + color_dict[path_county_id] + ";"
        if tooltips_dict != None and path_county_id in tooltips_dict:
            path[0].text += " - " + str(tooltips_dict[path_county_id])
    etree.ElementTree(root).write(target_loc, pretty_print=True)

    
def genCSSRGB(R, G, B):
    return "rgb(" + str(R) + "," + str(G) + "," + str(B) + ")"

In [12]:
# load in frc county data
frczips = pd.read_csv(frc_county_loc)
frczips.head()

,Team Number,Country,State,City,Postal Code,State Code,fips
0,1,USA,Michigan,Pontiac,48340,MI,c26125
1,4,USA,California,Van Nuys,91406,CA,c06037
2,8,USA,California,Palo Alto,94301,CA,c06085
3,11,USA,New Jersey,Flanders,07836,NJ,c34027
4,16,USA,Arkansas,Mountain Home,72653,AR,"c05005,c05089"


In [10]:
# start counting density, and keep track of max
maxSoFar = 0
countyDensity = {}
tooltips_dict = {}
for index, row in frczips.iterrows():
    fips = row["fips"].split(",")
    for fip in fips:
        if fip in countyDensity:
            countyDensity[fip] += 1
            tooltips_dict[fip] += ", " + str(row["Team Number"])
        else:
            countyDensity[fip] = 1
            tooltips_dict[fip] = str(row["Team Number"])
        maxSoFar = max([countyDensity[fip], maxSoFar])


In [11]:
# create color dictionary mapping each county name to a color
col_dict = {}
for fips_code in countyDensity:
    #for fips_code in fips_code_line.split(","):
        val = min_color_val + int((255.0 - min_color_val) * float(countyDensity[fips_code_line]) / float(maxSoFar))
        col_dict[fips_code] = genCSSRGB(255 - val, 255 - val, 255)

writeNewSVGWithColorDictionary(col_dict, tooltips_dict)